# Giochi di carte (carte piacentine)

## Classes

In [1]:
import random

In [2]:
class Carta():
    def __init__(self, seme, valore) -> None:
        self.figure_valori = {
            'Asso' : 1,
            'Fante' : 8,
            'Cavallo' : 9,
            'Re' : 10}
        self.valori_figure = {valore: figura for figura, valore in self.figure_valori.items()}

        self.seme = seme.capitalize()
        self.valore = self._converti_figura_in_valore(valore)

        self._check_seme()
        self._check_valore()

    def _check_seme(self):
        possible_semi = ['Bastoni', 'Denari', 'Spade', 'Coppe']
        if self.seme not in possible_semi:
            raise ValueError(f"Trovato {self.seme} come seme\n"
                              f"Il seme della carta deve essere per forza tra {possible_semi}")
                              
    def _check_valore(self):
        possible_valori = range(1,11)
        if self.valore not in possible_valori:
            raise ValueError(f"Trovato {self.valore} come valore\n"
                              f"Il valore della carta deve essere per forza tra {[i for i in possible_valori]}")

    def _converti_in_figura(self, valore):
        if valore > 7 or valore == 1:
            return self.valori_figure[valore]
        else:
            return str(valore)

    def _converti_figura_in_valore(self, valore):
        if isinstance(valore, str):
            return self.figure_valori[valore]
        else:
            return valore

    def __repr__(self) -> str:
        return f"{self._converti_in_figura(self.valore)} di {self.seme}"

In [140]:
class Mazzo():
    def __init__(self) -> None:
        semi = ['Bastoni', 'Denari', 'Spade', 'Coppe']
        valori = range(1,11)
                
        self.carte = {f'{seme[0]}{valore}' : Carta(seme, valore) for seme in semi for valore in valori}
        self.shuffle()

    def shuffle(self):
        keys =  list(self.carte.keys())
        random.shuffle(keys)
        self.carte = {key:self.carte[key] for key in keys}

    def togli_carta(self, carta):
        try:
            self.carte.pop(self.convert_carta_to_key(carta))
        except KeyError:
            print(f"{carta} gia' scartata")
    
    def pesca_carta(self):
        return self.carte.pop(list(self.carte.keys())[0])

    def convert_carta_to_key(self, carta):
        return f'{carta.seme[0]}{carta.valore}'

    def __repr__(self) -> str:
        return f"Mazzo con {len(self.carte)} carte"

In [141]:
mazzo = Mazzo()

In [123]:
class Giocatore():
    def __init__(self, nome) -> None:
        self.nome = nome
        self.carte_in_mano = []

    def pesca_carta(self, mazzo):
        self.carte_in_mano.append(mazzo.pesca_carta())

    def usa_carta(self, posizione_carta):
        return self.carte_in_mano.pop(posizione_carta)

    def __repr__(self) -> str:
        return f"{self.nome} ha {len(self.carte_in_mano)} carte in mano"

    

In [138]:
giocatore1 = Giocatore('Roberto')
giocatore1.pesca_carta(mazzo)
giocatore1.pesca_carta(mazzo)
giocatore1.pesca_carta(mazzo)
print(giocatore1.carte_in_mano)
giocatore1

[3 di Denari, Fante di Coppe, 4 di Bastoni]


Roberto ha 3 carte in mano

In [139]:
giocatore1.usa_carta(0)
giocatore1.carte_in_mano

[Fante di Coppe, 4 di Bastoni]

## Briscola

### Regole di gioco:

Una volta mischiato il mazzo ogni giocatore riceve tre carte, distribuite insieme ed in senso antiorario, viene poi scoperta la prima carta del mazzo che designerà il seme di Briscola. Quest'ultima viene messa per metà sotto al restante mazzo coperto cosicchè resterà sempre visibile ai giocatori durante lo svolgimento del round e sarà l'ultima carta ad essere pescata. Una volta distribuite le carte e designato il seme di Briscola ha inizio la partita.

Inizia il giocatore seduto alla destra di colui che ha distribuito le carte e si procede poi in senso antiorario. Ogni giocatore cercherà di aggiudicarsi la mano giocando la carta che ritiene più opportuna al fine di totalizzare il maggior numero di punti. Nella Briscola a 4 giocatori i punti vengono sommati tra i giocatori della stessa squadra. Colui che si aggiudica la mano inizierà il gioco nella mano successiva. Le regole della Briscola prevedono i seguenti criteri per aggiudicarsi una mano: Il primo giocatore a giocare una carta (detto giocatore di mano) determina con la stessa il seme di mano. I giocatori successivi non hanno alcun obbligo di giocare un particolare seme come invece avviene per giochi come il tressette. La mano è temporaneamente aggiudicata da un altro giocatore se questi posa sul tavolo una carta del seme di mano con valore di presa maggiore, o una carta del seme di Briscola anche se con valore di presa inferiore. Alla fine, quando tutti i giocatori avranno posato una carta sul tavolo, si aggiudicherà la mano colui che ha posato la carta di Briscola con valore di presa maggiore, o, in mancanza, da colui che ha giocato la carta del seme di mano con valore di presa maggiore; in mancanza di carte del seme di Briscola o di carte del seme di mano vince il primo giocatore di mano in quanto l'unico ad aver posato una carta del seme di mano. Essendo il seme di mano determinato di volta in volta dalla prima carta giocata, può coincidere col seme di Briscola ed in questo caso la mano è aggiudicata da chi gioca il seme di Briscola con valore di presa maggiore. Colui che si aggiudica la mano prende le carte giocate in quella mano e le pone coperte davanti a sé, prenderà poi una carta dal mazzo seguito dagli altri giocatori in senso antiorario.

In [4]:
from classes import Briscola, Mazzo

In [148]:
class Briscola():
    def __init__(self) -> None:
        self.mazzo = Mazzo()
        self.init_giocatori()

        self.dict_punti = {1: 11, 3: 10, 10: 4, 9: 3, 8: 2}

    def init_giocatori(self):
        self.numero_giocatori = int(input("In quanti volete giocare?"))
        self.contro_AI = False

        assert isinstance(self.numero_giocatori, int), "Non ho mai sentito mezze persone giocare a briscola. Metti un numero intero pls."

        if self.numero_giocatori == 1:
            contro_AI = input("Sei da solo: vuoi giocare contro l'AI? (s/n)")
            if contro_AI.lower()[0] == 's':
                print("Ottimo, allora in bocca al lupo!")
                self.contro_AI = True
            elif contro_AI.lower()[0] == 'n':
                print("Allora nada. Ricomincia il gioco se ti va.")
            else:
                raise ValueError("Accetto solo si o no come risposte.")

        elif self.numero_giocatori == 3:
            coppe_2 = self.mazzo.togli_carta(Carta("Coppe", 2))
            print(f"Dato che siete in 3 tolgo il 2 di Coppe, che e' la piu' inutile")
            
        elif self.numero_giocatori == 5:
            print("Mi dispiace ma non si puo' giocare per il momento a Briscola chiamata")
        
        else:
            print("Il numero massimo di giocatori e' 4")            

    def conta_punti(self):
        pass

    def __repr__(self) -> str:
        return f"Gioco di Briscola con {self.numero_giocatori} giocatori"

In [ ]:
class GiocatoreBriscola(Giocatore):
    def __init__(self, nome) -> None:
        super().__init__(nome)
        self.punti = 0
        self.dict_punti = {1: 11, 3: 10, 10: 4, 9: 3, 8: 2}
        self.carte_vinte = {}

    def aggiungi_carte_vinte(self, carte_vinte: list or dict):
        for carta in carte_vinte:
            self.carte_vinte

    def aggiungi_punti(self, punti):
        self.punti += punti

    def conta_punti(self):
        punti_totali = 0
        for key in self.carte_vinte.keys():
            try:
                punti_totali += self.dict_punti[int(key[1:])]
            except:
                pass
        return punti_totali

    

In [15]:
carta = mazzo.pesca_carta()

In [19]:
carta.key

AttributeError: 'Carta' object has no attribute 'key'

In [2]:
briscola = Briscola()

Dato che siete in 3 tolgo il 2 di Coppe, che e' la piu' inutile


## Scopa